 Set API Keys

In [1]:
import os
import os
os.environ["OPENAI_API_KEY"] = "you_api_key"
os.environ["COHERE_API_KEY"] = "you_api_key"

Import Necessary Libraries

In [2]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.retrievers import BM25Retriever
from langchain.schema import Document
from langchain.llms import OpenAI
import cohere


Load & Process Knowledge Base

In [3]:
# Load data
loader = TextLoader("learning_material.txt", encoding="utf-8")  # Replace with sample document
documents = loader.load()

# Split text into manageable chunks
text_splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=50)
docs = text_splitter.split_documents(documents)



Created a chunk of size 2269, which is longer than the specified 512


In [4]:
from langchain.embeddings import HuggingFaceEmbeddings

# Use Hugging Face Sentence Transformers (free, local) using openaiembeddings will get error
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Store embeddings in FAISS
vector_db = FAISS.from_documents(docs, embedding_model)
vector_db.save_local("faiss_index")

C:\Users\JunHong\AppData\Local\Temp\ipykernel_5028\2393451350.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
c:\Users\JunHong\OneDrive\Desktop\SIT\Year 2 Tri 2\LLM\Project_draft\llm_venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Implement Hybrid Retrieval (BM25 + FAISS)

In [9]:
# Load FAISS index
vector_db = FAISS.load_local("faiss_index", embedding_model, allow_dangerous_deserialization=True)
# need to set allow_dangerous_deserialization=True to load the index from disk
# otherwise will get error on security

# Initialize BM25 Retriever
bm25_retriever = BM25Retriever.from_documents(docs)

# Hybrid retrieval function
def hybrid_retrieve(query, top_k=5):
    """Retrieve documents using both BM25 and FAISS"""
    bm25_results = bm25_retriever.get_relevant_documents(query)[:top_k]
    faiss_results = vector_db.similarity_search(query, k=top_k)
    
    # Merge results, removing duplicates
    seen = set()
    combined_results = []
    for doc in bm25_results + faiss_results:
        if doc.page_content not in seen:
            combined_results.append(doc)
            seen.add(doc.page_content)
    
    return combined_results[:top_k]

# Test retrieval
query = "Explain decoder"
retrieved_docs = hybrid_retrieve(query)
for doc in retrieved_docs:
    print(doc.page_content[:300], "\n---")  # Show first 300 chars of each result


Abbreviations
AES Advanced Encryption Standard
AWGN additive white Gaussian noise
BC broadcast channel
BCC broadcast channel with confidential messages
BEC binary erasure channel
BSC binary symmetric channel
CA certification authority
DES Data Encryption Standard
DMC discrete memoryless channel
DMS  
---
Physical-Layer Security
From Information Theory to Security Engineering
This complete guide to physical-layer security presents the theoretical foundations, practical implementation, challenges, and benefits of a groundbreaking new model for secure
communication. Using a bottom-up approach from the  
---


Implement Re-Ranking with Cohere Rerank API

In [6]:
co = cohere.Client(os.getenv("COHERE_API_KEY"))

def rerank_results(query, retrieved_docs):
    """Re-rank retrieved documents using Cohere Rerank API"""
    rerank_inputs = [doc.page_content for doc in retrieved_docs]
    response = co.rerank(model="rerank-english-v2.0", query=query, documents=rerank_inputs, top_n=len(rerank_inputs))
    
    # Sort documents by relevance score
    ranked_docs = sorted(zip(response.results, retrieved_docs), key=lambda x: x[0].relevance_score, reverse=True)
    
    return [doc[1] for doc in ranked_docs]

# Apply re-ranking
reranked_docs = rerank_results(query, retrieved_docs)


Response Generation with GPT-4

In [10]:
# Initialize GPT-4 model
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.3)

def generate_response(query, reranked_docs):
    """Generate response using GPT-4 with retrieved knowledge"""
    context = "\n\n".join([doc.page_content for doc in reranked_docs])
    
    prompt = f"""You are a personalized NLP learning assistant. Use the provided knowledge to answer the question accurately.

    Knowledge Context:
    {context}
    
    Question: {query}
    Answer:
    """
    
    response = llm.predict(prompt)
    return response

# Generate response
final_response = generate_response(query, reranked_docs)
print(final_response)


A decoder is a device or algorithm that converts encoded data back into its original format. In the context of communication systems, particularly those involving encoding for secure transmission, a decoder plays a crucial role in ensuring that the intended receiver can accurately recover the original message from the received codeword.

In secure communication, messages are often encoded using specific coding schemes to protect against eavesdropping and to ensure reliable transmission over noisy channels. The encoding process transforms the original message (often referred to as the plaintext) into a codeword, which is then transmitted over the communication channel. The decoder, located at the receiving end, is responsible for interpreting the received codeword and reconstructing the original message.

Key functions of a decoder include:

1. **Error Correction**: Decoders often incorporate error-correcting techniques to handle noise and interference that may corrupt the transmitted c

In [ ]:
#this session is for debugging / testing the openai api, 
# DO NOT NEED to run if the code above works
# https://platform.openai.com/docs/api-reference/debugging-requests
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
    messages=[{
        "role": "user",
        "content": "Say this is a test",
    }],
    model="gpt-4o-mini",
)
print(response._request_id)
print(response)
print(os.getenv("OPENAI_API_KEY"))


Personalization (User-Adaptive Learning)

In [11]:
user_profile = {}  # Dictionary to store user preferences

def personalize_response(user_id, query, response):
    """Store responses for user history tracking"""
    if user_id not in user_profile:
        user_profile[user_id] = []
    
    user_profile[user_id].append({"query": query, "response": response})

# Store response for a user
personalize_response("user_123", query, final_response)

# Retrieve past user interactions
print("User Interaction History:", user_profile["user_123"])


User Interaction History: [{'query': 'Explain decoder', 'response': 'A decoder is a device or algorithm that converts encoded data back into its original format. In the context of communication systems, particularly those involving encoding for secure transmission, a decoder plays a crucial role in ensuring that the intended receiver can accurately recover the original message from the received codeword.\n\nIn secure communication, messages are often encoded using specific coding schemes to protect against eavesdropping and to ensure reliable transmission over noisy channels. The encoding process transforms the original message (often referred to as the plaintext) into a codeword, which is then transmitted over the communication channel. The decoder, located at the receiving end, is responsible for interpreting the received codeword and reconstructing the original message.\n\nKey functions of a decoder include:\n\n1. **Error Correction**: Decoders often incorporate error-correcting tec